In [16]:
import pandas as pd
import numpy as np

In [17]:
faostat_production = pd.read_csv("X:/cadlan/stream_2/Step4/REV_fao_stat_production_mov_average_v2.csv")
grapc = pd.read_csv("X:/cadlan/stream_2/Step5/GRAPC_production_region_melted.csv")
country_code = pd.read_excel("Z:/achaidir/Country Grids/ISO-3166-Country-Code_Final.xlsx")

In [18]:
faostat_production

,Area,Year,IMAGE Classification,Value
0,Afghanistan,1970,Vegetables & fruits,1298080.000
1,Algeria,1970,Vegetables & fruits,2385004.200
2,Argentina,1970,Vegetables & fruits,7118646.200
3,Armenia,1970,Vegetables & fruits,0.000
4,Australia,1970,Vegetables & fruits,2755132.600
...,...,...,...,...
26285,Equatorial Guinea,2000,Plant based fibres,414.216
26286,Equatorial Guinea,2005,Plant based fibres,345.922
26287,Equatorial Guinea,2010,Plant based fibres,269.728
26288,Equatorial Guinea,2015,Plant based fibres,238.618


In [19]:
faostat_production = faostat_production.rename(columns={"Value": "FAO Production", "IMAGE Classification": "NGFBFC", "Year": "time"})
faostat_production_merge = pd.merge(left=faostat_production, right=country_code, left_on='Area', right_on='ISO Country')
faostat_production_columns = faostat_production_merge[['IMAGE Region Name', 'time', 'NGFBFC', 'FAO Production']]
faostat_production_columns = faostat_production_columns.groupby(['IMAGE Region Name', 'time', 'NGFBFC']).sum()
faostat_production_index = faostat_production_columns.reset_index()
faostat_production_index.head()

,IMAGE Region Name,time,NGFBFC,FAO Production
0,Brazil,1970,Maize,13748855.0
1,Brazil,1970,Oil & palm fruit,37000.0
2,Brazil,1970,Other non-food & luxury & spices,1887902.6
3,Brazil,1970,Other temperate cereals,81692.0
4,Brazil,1970,Plant based fibres,2395394.2


In [20]:
faostat_production_index.to_excel("X:/cadlan/stream_2/Step4/REV_fao_stat_area_harvested_mov_average_v2_REGION.xlsx", index=False)

In [23]:
faostat_production_index['IMAGE Region Name'].unique()

array(['Brazil', 'C. Europe', 'Canada', 'China', 'E. Africa', 'India',
       'Indonesia', 'Japan', 'Kazakhstan region', 'Korea', 'Mexico',
       'Middle East', 'N. Africa', 'Oceania', 'Rest C. America',
       'Rest S. Africa', 'Rest S. America', 'Rest S. Asia', 'Russia',
       'SE. Asia', 'South Africa', 'Turkey', 'USA', 'Ukraine region',
       'W. Africa', 'W. Europe'], dtype=object)

In [24]:
faostat_production_index['NGFBFC'].unique()

array(['Maize', 'Oil & palm fruit', 'Other non-food & luxury & spices',
       'Other temperate cereals', 'Plant based fibres', 'Pulses', 'Rice',
       'Soybeans', 'Sugar crops', 'Temperate oil crops',
       'Temperate roots & tubers', 'Tropical cereals',
       'Tropical oil crops', 'Tropical roots & tubers',
       'Vegetables & fruits', 'Wheat'], dtype=object)

In [25]:
faostat_production_index[(faostat_production_index['IMAGE Region Name'] == "China") & (faostat_production_index['NGFBFC'] == 'Maize')]

,IMAGE Region Name,time,NGFBFC,FAO Production
462,China,1970,Maize,3.175021e+07
478,China,1975,Maize,4.537167e+07
494,China,1980,Maize,5.977909e+07
510,China,1985,Maize,7.138741e+07
526,China,1990,Maize,8.983375e+07
542,China,1995,Maize,1.095317e+08
558,China,2000,Maize,1.204216e+08
574,China,2005,Maize,1.380162e+08
590,China,2010,Maize,1.812608e+08
606,China,2015,Maize,2.445251e+08


In [26]:
grapc[(grapc['IMAGE Region Name'] == "China") & (grapc['NGFBFC'] == 'Maize')]

,IMAGE Region Name,NGFBFC,time,value
528,China,Maize,1970,3.350984e+11
529,China,Maize,1975,4.395404e+11
530,China,Maize,1980,5.393159e+11
531,China,Maize,1985,5.783373e+11
532,China,Maize,1990,6.601457e+11
533,China,Maize,1995,7.678565e+11
534,China,Maize,2000,7.791883e+11
535,China,Maize,2005,8.268485e+11
536,China,Maize,2010,9.565133e+11
537,China,Maize,2015,1.001221e+12


In [28]:
grapc = grapc.rename(columns={"GFRAC": "GRAPC"})
grapc.head()

,IMAGE Region Name,NGFBFC,time,GRAPC
0,Brazil,Maize,1970,1.835202e+11
1,Brazil,Maize,1975,2.087353e+11
2,Brazil,Maize,1980,2.253584e+11
3,Brazil,Maize,1985,2.562166e+11
4,Brazil,Maize,1990,2.755364e+11


In [30]:
grapc.columns

Index(['IMAGE Region Name', 'NGFBFC', 'time', 'GRAPC'], dtype='object')

In [32]:
grapc_copy = grapc.copy()
grapc_copy.loc[:,'FAO'] = 0.0
grapc_copy['FAO'] = grapc.groupby(['NGFBFC', 'time', 'IMAGE Region Name'])['GRAPC'].transform(lambda x: faostat_production_index[(faostat_production_index['NGFBFC'] == x.name[0]) & (faostat_production_index['time'] == x.name[1]) & (faostat_production_index['IMAGE Region Name'] == x.name[2])]['FAO Production'].sum())

In [35]:
grapc_copy.loc[:, 'selisih'] = abs(grapc_copy['GRAPC'] - grapc_copy['FAO'])

In [37]:
grapc_copy[(grapc_copy['IMAGE Region Name'] == "China") & (grapc_copy['NGFBFC'] == 'Maize')]

,IMAGE Region Name,NGFBFC,time,GRAPC,FAO,selisih
528,China,Maize,1970,3.350984e+11,3.175021e+07,3.350666e+11
529,China,Maize,1975,4.395404e+11,4.537167e+07,4.394951e+11
530,China,Maize,1980,5.393159e+11,5.977909e+07,5.392561e+11
531,China,Maize,1985,5.783373e+11,7.138741e+07,5.782659e+11
532,China,Maize,1990,6.601457e+11,8.983375e+07,6.600559e+11
533,China,Maize,1995,7.678565e+11,1.095317e+08,7.677470e+11
534,China,Maize,2000,7.791883e+11,1.204216e+08,7.790679e+11
535,China,Maize,2005,8.268485e+11,1.380162e+08,8.267105e+11
536,China,Maize,2010,9.565133e+11,1.812608e+08,9.563320e+11
537,China,Maize,2015,1.001221e+12,2.445251e+08,1.000976e+12


In [38]:
grapc_copy.sort_values(by='selisih', ascending=False)

,IMAGE Region Name,NGFBFC,time,GRAPC,FAO,selisih
2024,Middle East,Sugar crops,1970,1.893548e+13,4856909.6,1.893547e+13
98,Brazil,Sugar crops,2020,1.450097e+13,739441045.2,1.450023e+13
96,Brazil,Sugar crops,2010,1.447834e+13,701890693.6,1.447764e+13
91,Brazil,Sugar crops,1985,1.420824e+13,238500256.0,1.420800e+13
97,Brazil,Sugar crops,2015,1.395918e+13,756345913.4,1.395842e+13
...,...,...,...,...,...,...
492,Canada,Tropical oil crops,2010,0.000000e+00,0.0,0.000000e+00
493,Canada,Tropical oil crops,2015,0.000000e+00,0.0,0.000000e+00
484,Canada,Tropical oil crops,1970,0.000000e+00,0.0,0.000000e+00
487,Canada,Tropical oil crops,1985,0.000000e+00,0.0,0.000000e+00


In [39]:
grapc_copy.to_excel("X:/cadlan/stream_2/Step5/Komparasi_FAO_GRAPC.xlsx", index=False)